In [43]:
import casadi as ca

In [44]:
num_poles = 3

pole_as = [0.05, 0.1, 0.15]
pole_ls = [0.1, 0.2, 0.3]
pole_ms = [0.1, 0.15, 0.2]
pole_ds = [0.01, 0.01, 0.01]
pole_Js = [0.13, 0.12, 0.11]

m_c = 0.5
g = 9.81

In [45]:
s = ca.MX.sym("s")
d_s = ca.MX.sym("d_s")
dd_s = ca.MX.sym("dd_s")
thetas = [ca.MX.sym(f"theta_{i+1}") for i in range(num_poles)]
d_thetas = [ca.MX.sym(f"d_theta_{i+1}") for i in range(num_poles)]
dd_thetas = [ca.MX.sym(f"dd_theta_{i+1}") for i in range(num_poles)]
u = dd_s

derivate_by_time_dict = {s: d_s, d_s: dd_s}
for i in range(num_poles):
    derivate_by_time_dict[thetas[i]] = d_thetas[i]
    derivate_by_time_dict[d_thetas[i]] = dd_thetas[i]

def derivate_by_time(expr):
    for var, der in list(derivate_by_time_dict.items())[::-1]:
        expr = ca.substitute(expr, var, der)
    return expr

In [46]:
pole_pcs = []
for i, (theta, a) in enumerate(zip(thetas, pole_as)):
    prev_1 = 0
    prev_2 = 0
    for prev_l, prev_theta in list(zip(pole_ls, thetas))[:i]:
        prev_1 += -prev_l*ca.sin(prev_theta)
        prev_2 += prev_l*ca.cos(prev_theta)
    pole_pcs.append([s-a*ca.sin(theta)+prev_1, a*ca.cos(theta)+prev_2])

In [47]:
def square_sum(x: tuple):
    result = 0
    for x_i in x:
        result += x_i**2
    return result

T_c = 1/2*m_c*d_s**2
T_p = 1/2*sum(m*square_sum(pc) for m, pc in zip(pole_ms, pole_pcs))+1/2*sum(J*d_theta**2 for J, d_theta in zip(pole_Js, d_thetas))
T = T_c + T_p
V = 0
for m, pc in zip(pole_ms, pole_pcs):
    V += g*m*pc[0]
R = 0
prev_w = 0
for d, d_theta in zip(pole_ds, d_thetas):
    R += 1/2*d*(d_theta-prev_w)**2
    prev_w = d_theta

In [48]:
L = T-V
eqs = []
lh = derivate_by_time(ca.jacobian(L, d_s))
lh -= ca.jacobian(L, s) + ca.jacobian(R, d_s)
rh = u
eqs.append(lh-rh)
for theta, d_theta in zip(thetas, d_thetas):
    lh = derivate_by_time(ca.jacobian(L, d_theta))
    lh -= ca.jacobian(L, theta) + ca.jacobian(R, d_theta)
    rh = 0
    eqs.append(lh-rh)
eqs

[SX(@1=0.1, @2=(s-(0.05*sin(theta_1))), @3=0.15, @4=-0.1, @5=((s-(@1*sin(theta_2)))+(@4*sin(theta_1))), @6=((s-(@3*sin(theta_3)))+((@4*sin(theta_1))+(-0.2*sin(theta_2)))), (((0.25*(dd_s+dd_s))-((0.5*(((@1*(@2+@2))+(@3*(@5+@5)))+(0.2*(@6+@6))))+-4.4145))-dd_s)),
 SX(@1=0.5, @2=0.15, @3=0.1, @4=-0.1, @5=((s-(@3*sin(theta_2)))+(@4*sin(theta_1))), @6=(@4*cos(theta_1)), @7=((@3*cos(theta_2))+(@3*cos(theta_1))), @8=0.05, @9=(s-(@8*sin(theta_1))), @10=(@8*cos(theta_1)), @11=(@8*cos(theta_1)), @12=0.2, @13=((s-(@2*sin(theta_3)))+((@4*sin(theta_1))+(-0.2*sin(theta_2)))), @14=(@4*cos(theta_1)), @15=((@2*cos(theta_3))+((@3*cos(theta_1))+(@12*cos(theta_2)))), @16=0.005, @17=(d_theta_2-d_theta_1), ((@1*(0.13*(dd_theta_1+dd_theta_1)))-(((@1*(((@2*(((@5+@5)*@6)-((@7+@7)*(@3*sin(theta_1)))))-(@3*(((@9+@9)*@10)+((@11+@11)*(@8*sin(theta_1))))))+(@12*(((@13+@13)*@14)-((@15+@15)*(@3*sin(theta_1)))))))-(((1.4715*@6)-(0.981*@10))+(1.962*@14)))+((@16*(d_theta_1+d_theta_1))-(@16*(@17+@17)))))),
 SX(@1=0.5, @2

In [49]:
ca.fsolve

0x1
